In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()

٢٢/٠٧/١٣ ٢٣:١٢:٤٨ WARN Utils: Your hostname, mohand-Lenovo-ideapad-310-15IKB resolves to a loopback address: 127.0.1.1; using 192.168.1.17 instead (on interface wlp2s0)
٢٢/٠٧/١٣ ٢٣:١٢:٤٨ WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
٢٢/٠٧/١٣ ٢٣:١٢:٤٨ WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Create the schema of the streamed files (check the column names and types from the CSV files)

In [3]:
from pyspark.sql.types import (StructType, StructField,
                               StringType, IntegerType,DoubleType)

recordSchema = StructType([StructField('_c0', IntegerType(), True),
                           StructField('Date', StringType(), True),
                           StructField('Open', DoubleType(), True),
                           StructField('High', DoubleType(), True),
                           StructField('Low', DoubleType(), True),
                           StructField('Close',DoubleType(),True),
                           StructField('Adj Close',DoubleType(),True),
                          StructField('Volume',IntegerType(),True)])

### Create the dataframe by reading the stream using format "csv" and the schema you created.

In [4]:
df = spark.readStream.format("csv") \
    .schema(recordSchema) \
    .load("kospi/")

### Make sure the sataframe is streaming the files from the folder

In [5]:
df.isStreaming

True

### Create a stream writer into memory and specify the query name "stock:

In [6]:
writer = df.writeStream.outputMode("append") \
    .format("memory")  \
    .queryName('stock')

### Start the write stream and make sure it works (read all columns from the table)

In [7]:
query= writer.start()

٢٢/٠٧/١٣ ٢٣:١٢:٥٤ WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-9df92afc-a15f-414e-bbcb-546eee8d22d9. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
٢٢/٠٧/١٣ ٢٣:١٢:٥٤ WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [8]:
spark.sql('select * from stock').show(2)

+---+----+----+----+---+-----+---------+------+
|_c0|Date|Open|High|Low|Close|Adj Close|Volume|
+---+----+----+----+---+-----+---------+------+
+---+----+----+----+---+-----+---------+------+



### Remove the first row from the data (hint: drop the rows where ALL values are null), then add a new column "diff", which is the difference between high and low columns

In [9]:
spark.sql("select * from stock where Open is not null").show(3)

+---+----+----+----+---+-----+---------+------+
|_c0|Date|Open|High|Low|Close|Adj Close|Volume|
+---+----+----+----+---+-----+---------+------+
+---+----+----+----+---+-----+---------+------+



In [10]:
df=df.dropna()

In [11]:
from pyspark.sql.functions import *

In [12]:
df=df.withColumn('diff',col('High')-col('Low'))

In [13]:
query.stop()

٢٢/٠٧/١٣ ٢٣:١٢:٥٧ ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@7ee01d71 is aborting.
٢٢/٠٧/١٣ ٢٣:١٢:٥٧ ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@7ee01d71 aborted.


### Create a new write stream using the new generated dataframe and call the generate table "modified_data"

In [14]:
writer2 = df.writeStream.outputMode("append") \
    .format("memory")  \
    .queryName('modified_data')

In [15]:
query=writer2.start()

٢٢/٠٧/١٣ ٢٣:١٢:٥٧ WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-cb2afbb0-ff4b-4882-8963-9c1ed9b768bc. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
٢٢/٠٧/١٣ ٢٣:١٢:٥٧ WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [16]:
spark.sql('select * from modified_data').show()

+---+----+----+----+---+-----+---------+------+----+
|_c0|Date|Open|High|Low|Close|Adj Close|Volume|diff|
+---+----+----+----+---+-----+---------+------+----+
+---+----+----+----+---+-----+---------+------+----+



### Write the generated data into files instead of the memory. 

In [17]:
writer = df.writeStream.outputMode("append")\
.format("parquet")\
.option("path", "Generated/")\
.option("checkpointLocation", "chkpnt") 

In [18]:
query=writer.start()

٢٢/٠٧/١٣ ٢٣:١٢:٥٨ WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [19]:
query.stop()

### Stop the query. Now, try reading the generated parquet files into a normal dataframe
- Create a schema and use it to read the data.
- Show the output.

In [28]:
finalDF= spark.read.format("csv") \
    .schema(recordSchema) \
    .load("Generated/")

In [29]:
finalDF.show()

+---+----------+------------+------------+------------+------------+------------+------+
|_c0|      Date|        Open|        High|         Low|       Close|   Adj Close|Volume|
+---+----------+------------+------------+------------+------------+------------+------+
|240|2000-12-05|26585.300781|27367.300781|26372.099609|27011.800781|25526.091797| 91019|
|241|2000-12-06|27011.800781|27509.400391|26798.599609|26869.699219|25391.804688|105791|
|242|2000-12-07|27011.800781|27011.800781|26478.699219|26656.400391|25190.236328| 40656|
|243|2000-12-08|26656.400391|27722.699219|26656.400391|27651.599609|26130.699219|149964|
|244|2000-12-11|27687.099609|     28860.0|27651.599609|28078.099609|26533.740234|159671|
|245|2000-12-12|28042.599609|28078.099609|27438.300781|27935.900391|26399.361328| 74560|
|246|2000-12-13|27651.599609|     29286.5|27651.599609|28469.099609|26903.234375|270385|
|247|2000-12-14|28469.099609|29784.099609|28291.300781|28362.400391| 26802.40625|256317|
|248|2000-12-15|28362

### Sort the dataframe based on the ID

In [30]:
finalDFSorted = finalDF.sort('_c0')
finalDFSorted.show()

+---+----------+------------+------------+------------+------------+------------+------+
|_c0|      Date|        Open|        High|         Low|       Close|   Adj Close|Volume|
+---+----------+------------+------------+------------+------------+------------+------+
|  0|2000-01-04|22817.900391|25696.800781|22817.900391|24879.300781|23510.880859|108745|
|  1|2000-01-05|24523.900391|26229.900391|23670.900391|24417.300781|23074.294922|175990|
|  2|2000-01-06|24381.699219|24666.099609|22746.800781|22817.900391|21562.865234| 71746|
|  3|2000-01-07|     22036.0|24879.300781|     22036.0|23884.199219|22570.513672|120984|
|  4|2000-01-10|24879.300781|25519.099609|23813.099609|24061.900391|22738.439453|151371|
|  5|2000-01-11|     24168.5|     25021.5|23955.199219|24239.599609|22906.365234| 95943|
|  6|2000-01-12|     24168.5|24452.800781|23457.599609|23670.900391|22368.947266| 61899|
|  7|2000-01-13|23670.900391|24132.900391|23102.199219|23244.400391| 21965.90625| 57538|
|  8|2000-01-14|23457